In [ ]:
import code.io as io
model = io.read_yeast_model() # loading
from cobra import Reaction, Metabolite, Gene

Adding the metabolites:

In [ ]:
Beta_farnesene = Metabolite(name='Beta-farnesene',id='beta_farnesene_c',compartment='c')

Adding the gene for beta-farnesene synthase:

In [ ]:
Beta_farnesene_synthase_gene = Gene(id='R08695',name='G_R08695',functional=True,)

Adding the reaction for beta-farnesene and the demand reaction for beta-farnesene

In [ ]:
from cobra import Reaction, Metabolite, Gene

Beta_farnesene_synthase_reaction = Reaction('R08695')

Beta_farnesene_synthase_reaction.add_metabolites({model.metabolites.s_0190: -1,
                                                  model.metabolites.s_0633: 1,
                                                  Beta_farnesene: 1 # our product
                                                  })

Beta_farnesene_synthase_reaction

model.add_reactions([Beta_farnesene_synthase_reaction])

model.reactions.get_by_id(Beta_farnesene_synthase_reaction.id)
Beta_farnesene_demand_reaction = Reaction('BFD')
Beta_farnesene_demand_reaction.add_metabolites({
                                                  Beta_farnesene: -1 # our product
                                                  })
Beta_farnesene_demand_reaction
model.add_reactions([Beta_farnesene_demand_reaction])
model.reactions.get_by_id(Beta_farnesene_demand_reaction.id)

